<center><span style="font-size:28px; font-weight:bold;">Implementez un modèle de scoring</span><center>

<center><span style="font-size:28px; font-weight:bold;">2 - Modélisation</span><center>

# 1 - Importation des librairies et des données

## 1.1 - Importation des différentes librairies

In [1]:
import mlflow
import mlflow.sklearn

# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Autres bibliothèques utiles :
import time

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import cross_validate

In [2]:
# On va démarrer le chronomètre :
start = time.time()

## 1.2 - Importation des données

In [3]:
# Initialiser MLFlow
mlflow.set_experiment("Home Credit Default Risk - Modelisation")

<Experiment: artifact_location='file:///Users/Gary/Documents/GitHub/P7_implementer_modele_scoring/mlruns/792830381958516671', creation_time=1726122940119, experiment_id='792830381958516671', last_update_time=1726122940119, lifecycle_stage='active', name='Home Credit Default Risk - Modelisation', tags={}>

In [4]:
# Spécifions les différents chemins possibles
directories = [
    "C:\\Users\\mauge\\Documents\\github\\P7_implementer_modele_scoring",
    "/Users/Gary/Documents/GitHub/FormationData/DataScientist_P7/Projet+Mise+en+prod+-+home-credit-default-risk",
    "/Users/Gary/Documents/GitHub/P7_implementer_modele_scoring",
    "C:\\Users\\mauge\\OneDrive\\Bureau\\Projet+Mise+en+prod+-+home-credit-default-risk"
]

In [5]:
def load_csv(file_name, directory_options, **kwargs):
    for directory in directory_options:
        file_path = os.path.join(directory, file_name)
        if os.path.exists(file_path):
            print(f"Loading {file_name} from {file_path}")
            return pd.read_csv(file_path, **kwargs)
    print(f"{file_name} not found in any of the specified directories.")
    return None

# Chargement des fichiers
#df_application_train = load_csv('df_final.csv', directories)
X_train = load_csv('X_train.csv', directories)
y_train = load_csv('y_train.csv', directories).values.ravel()
X_test = load_csv('X_test.csv', directories)
y_test = load_csv('y_test.csv', directories).values.ravel()

Loading X_train.csv from /Users/Gary/Documents/GitHub/P7_implementer_modele_scoring/X_train.csv
Loading y_train.csv from /Users/Gary/Documents/GitHub/P7_implementer_modele_scoring/y_train.csv
Loading X_test.csv from /Users/Gary/Documents/GitHub/P7_implementer_modele_scoring/X_test.csv
Loading y_test.csv from /Users/Gary/Documents/GitHub/P7_implementer_modele_scoring/y_test.csv


In [6]:
X_train.head()

,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,0.0,0.0,1.0,0.0,0.001512,0.090287,0.090032,0.077441,0.256702,0.687292,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.002089,0.311736,0.132924,0.271605,0.045506,0.832143,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,1.0,1.0,0.0,0.000358,0.022472,0.020025,0.023569,0.135340,0.877380,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.000935,0.066837,0.109477,0.063973,0.107480,0.876567,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.000819,0.116854,0.078975,0.117845,0.393191,0.894951,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 2 - Premier modèle de prédiction

## 2.1 - Régression logistique

Cette régression logistique va nous permettre d'avoir une base.

In [7]:
# Modèle simple
model = LogisticRegression()

# Test rapide avec seulement 2 folds
with mlflow.start_run(run_name="Logistic Regression Test"):
    scores = cross_validate(model, X_train, y_train.ravel(), cv=2, 
                            scoring={'Accuracy': 'accuracy'},
                            return_train_score=False, n_jobs=-1)
    
    # Calcul de la moyenne d'Accuracy
    accuracy_mean = np.mean(scores['test_Accuracy'])
    fit_time_mean = np.mean(scores['fit_time'])
    score_time_mean = np.mean(scores['score_time'])

    # Logging dans MLFlow
    mlflow.log_param("Model", "Logistic Regression")
    mlflow.log_metric("Accuracy_mean", accuracy_mean)
    mlflow.log_metric("Fit_time_mean", fit_time_mean)
    mlflow.log_metric("Score_time_mean", score_time_mean)
    
    # Enregistrement du modèle dans MLFlow
    mlflow.sklearn.log_model(model, "model")

# Résultat rapide
print(f"Accuracy_mean: {accuracy_mean}")

/Users/Gary/opt/anaconda3/envs/DataScientist_Projet_7/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/Gary/opt/anaconda3/envs/DataScientist_Projet_7/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https:/

Accuracy_mean: 0.919371990049972


In [ ]:
mlflow.end_run()

In [7]:
#mlflow.start_run(run_name="Modélisation 1")

In [8]:
# Définition des modèles :
models = {
    'Régression logistique': LogisticRegression(max_iter=1000)
}

In [ ]:
# Initialisation d'un DataFrame pour stocker les résultats
results_df = pd.DataFrame(columns=['Model', 'AUC_mean', 'Accuracy_mean', 'Fit_time_mean', 'Score_time_mean'])

# Évaluation de chaque modèle avec cross_validate
for name, model in models.items():
    with mlflow.start_run(run_name=name):
        scores = cross_validate(model, X_train, y_train, cv=5, 
                                scoring={'AUC': 'roc_auc', 'Accuracy': 'accuracy'},
                                return_train_score=False, n_jobs=-1)
        
        # Calcul des moyennes
        auc_mean = np.mean(scores['test_AUC'])
        accuracy_mean = np.mean(scores['test_Accuracy'])
        fit_time_mean = np.mean(scores['fit_time'])
        score_time_mean = np.mean(scores['score_time'])

        # Logging dans MLFlow
        mlflow.log_param("Model", name)
        mlflow.log_metric("AUC_mean", auc_mean)
        mlflow.log_metric("Accuracy_mean", accuracy_mean)
        mlflow.log_metric("Fit_time_mean", fit_time_mean)
        mlflow.log_metric("Score_time_mean", score_time_mean)
        
        # Enregistrement du modèle dans MLFlow
        mlflow.sklearn.log_model(model, "model")
        
        # Stockage dans DataFrame des résultats
        new_row = pd.DataFrame({
            'Model': [name],
            'AUC_mean': [auc_mean],
            'Accuracy_mean': [accuracy_mean],
            'Fit_time_mean': [fit_time_mean],
            'Score_time_mean': [score_time_mean]
        })

        results_df = pd.concat([results_df, new_row], ignore_index=True)

# Affichage des résultats :
print(results_df)

In [8]:
mlflow.end_run()

## 2.2 - Régression Logistique après SMOTE

L'utilisation de **SMOTE** (*Synthetic Minority Over-sampling Technique*) est particulièrement utile lorsque l'on travaille avec des ensembles de données déséquilibrés. En data science, un ensemble de données est dit déséquilibré lorsque la répartition des classes dans la variable cible n'est pas équilibrée, c'est-à-dire que l'une des classes (par exemple, les cas de défaut de paiement) est beaucoup moins représentée que l'autre (les non-défauts).

**Pourquoi utiliser SMOTE ?**

1. **Déséquilibre des classes** : Lorsque les classes sont déséquilibrées, les algorithmes de machine learning ont tendance à biaiser leurs prédictions en faveur de la classe majoritaire. Par exemple, dans le cas d’un modèle de scoring de crédit, si les défauts de paiement (classe minoritaire) représentent seulement 5 % des observations, un modèle pourrait simplement prédire que tout le monde remboursera son prêt pour obtenir une haute précision (en ignorant totalement les 5 % de défauts).
   
2. **SMOTE pour rééquilibrer les classes** : SMOTE génère de nouvelles observations synthétiques pour la classe minoritaire en combinant plusieurs exemples existants proches dans l’espace des caractéristiques. Cela permet de rééquilibrer le jeu de données sans simplement dupliquer les observations, ce qui pourrait mener à de l'overfitting.

3. **Amélioration des performances du modèle** :
   - **Meilleure sensibilité aux classes minoritaires** : Grâce à SMOTE, notre modèle pourra mieux identifier les cas appartenant à la classe minoritaire (dans notre cas, les clients à risque de défaut de paiement).
   - **Précision des prédictions** : Cela évite au modèle de donner une précision trompeuse en prédisant principalement la classe majoritaire. L'utilisation de métriques comme l'**AUC-ROC** ou le **F1-Score** devient également plus pertinente car elles tiennent compte des performances sur les deux classes.
   
4. **Limitation du biais du modèle** : Sans une correction comme SMOTE, le modèle risque de négliger la classe minoritaire et de mal gérer les faux négatifs (prédire qu'un client remboursera alors qu'il va faire défaut). Dans le contexte d'un modèle de scoring crédit, cela peut avoir des conséquences importantes sur le coût métier.

**Comment fonctionne SMOTE ?**

SMOTE fonctionne en trois étapes :
1. **Sélection des points de la classe minoritaire**.
2. **Création des observations synthétiques** : Pour chaque point de la classe minoritaire, SMOTE sélectionne les k plus proches voisins (en général 5). Ensuite, il génère un point synthétique sur la ligne qui relie l'exemple de la classe minoritaire sélectionné et l'un de ses voisins.
3. **Ajout des nouvelles données au jeu d'entraînement** : Les observations synthétiques sont ajoutées au jeu d'entraînement, augmentant la proportion de la classe minoritaire.

In [ ]:
# Application de SMOTE sur les données d'entraînement
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [ ]:
# Définition des modèles :
models = {
    'Régression logistique après SMOTE': LogisticRegression(max_iter=1000)
}

In [ ]:
# Initialisation d'un DataFrame pour stocker les résultats
results_df = pd.DataFrame(columns=['Model', 'AUC_mean', 'Accuracy_mean', 'Fit_time_mean', 'Score_time_mean'])

# Évaluation de chaque modèle avec cross_validate
for name, model in models.items():
    with mlflow.start_run(run_name=name):
        scores = cross_validate(model, X_train_smote, y_train_smote, cv=5, 
                                scoring={'AUC': 'roc_auc', 'Accuracy': 'accuracy'},
                                return_train_score=False, n_jobs=-1)
        
        # Calcul des moyennes
        auc_mean = np.mean(scores['test_AUC'])
        accuracy_mean = np.mean(scores['test_Accuracy'])
        fit_time_mean = np.mean(scores['fit_time'])
        score_time_mean = np.mean(scores['score_time'])

        # Logging dans MLFlow
        mlflow.log_param("Model", name)
        mlflow.log_metric("AUC_mean", auc_mean)
        mlflow.log_metric("Accuracy_mean", accuracy_mean)
        mlflow.log_metric("Fit_time_mean", fit_time_mean)
        mlflow.log_metric("Score_time_mean", score_time_mean)
        
        # Enregistrement du modèle dans MLFlow
        mlflow.sklearn.log_model(model, "model")
        
        # Stockage dans DataFrame des résultats
        new_row = pd.DataFrame({
            'Model': [name],
            'AUC_mean': [auc_mean],
            'Accuracy_mean': [accuracy_mean],
            'Fit_time_mean': [fit_time_mean],
            'Score_time_mean': [score_time_mean]
        })

        results_df = pd.concat([results_df, new_row], ignore_index=True)

# Affichage des résultats :
print(results_df)

In [ ]:
mlflow.end_run()

In [ ]:
elapsed_time = time.time() - start  # Calcule le temps écoulé
print(f"Temps total d'exécution du notebook : {elapsed_time:.2f} secondes.")